[620.Not Boring Movies](https://leetcode.com/problems/not-boring-movies/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
import pandas as pd

data = [[1, 'War', 'great 3D', 8.9], [2, 'Science', 'fiction', 8.5], [3, 'irish', 'boring', 6.2],
        [4, 'Ice song', 'Fantacy', 8.6], [5, 'House card', 'Interesting', 9.1]]
cinema = pd.DataFrame(data, columns=['id', 'movie', 'description', 'rating']).astype(
    {'id': 'Int64', 'movie': 'object', 'description': 'object', 'rating': 'Float64'})

In [0]:
cinema

,id,movie,description,rating
0,1,War,great 3D,8.9
1,2,Science,fiction,8.5
2,3,irish,boring,6.2
3,4,Ice song,Fantacy,8.6
4,5,House card,Interesting,9.1


In [0]:
# STEP 1
filter_cinema = cinema[(cinema['id'] % 2 !=0 ) & (cinema['description']!= 'boring')]

result = filter_cinema.sort_values(by='rating',ascending=False)
display(result)

id,movie,description,rating
5,House card,Interesting,9.1
1,War,great 3D,8.9


In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

In [0]:
spark_cinema_df = spark.createDataFrame(cinema)
spark_cinema_df.show()

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  1|       War|   great 3D|   8.9|
|  2|   Science|    fiction|   8.5|
|  3|     irish|     boring|   6.2|
|  4|  Ice song|    Fantacy|   8.6|
|  5|House card|Interesting|   9.1|
+---+----------+-----------+------+



In [0]:
spark_cinema_df\
            .filter((F.col('description')!='boring') & (F.col('id')%2!=0))\
            .orderBy(F.col('rating').desc())\
            .show()

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



In [0]:
spark_cinema_df.createTempView('cinemass')

spark.sql(
    '''
    SELECT *
    FROM cinemass
    WHERE description!='boring' and id%2!= 0
    ORDER BY rating desc;
    '''
).show()

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



[1251. Average Selling Price](https://leetcode.com/problems/average-selling-price/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:

data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15],
        [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype(
    {'product_id': 'Int64', 'start_date': 'datetime64[ns]', 'end_date': 'datetime64[ns]', 'price': 'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype(
    {'product_id': 'Int64', 'purchase_date': 'datetime64[ns]', 'units': 'Int64'})

In [0]:
prices

,product_id,start_date,end_date,price
0,1,2019-02-17,2019-02-28,5
1,1,2019-03-01,2019-03-22,20
2,2,2019-02-01,2019-02-20,15
3,2,2019-02-21,2019-03-31,30


In [0]:
units_sold

,product_id,purchase_date,units
0,1,2019-02-25,100
1,1,2019-03-01,15
2,2,2019-02-10,200
3,2,2019-03-22,30


In [0]:
merged = pd.merge_asof(
    units_sold.sort_values('purchase_date'),
    prices.sort_values('start_date'),
    by='product_id',
    left_on='purchase_date',
    right_on='start_date',
    direction='backward'
)

# Filter only rows where purchase_date is within the price interval
merged = merged[merged['purchase_date'] <= merged['end_date']]

# Calculate revenue per row
merged['revenue'] = merged['units'] * merged['price']

# Group by product_id and compute average selling price
result = merged.groupby('product_id').agg(
    total_revenue=('revenue', 'sum'),
    total_units=('units', 'sum')
).reset_index()

result['average_price'] = result['total_revenue'] / result['total_units']
result = result[['product_id', 'average_price']]
result['average_price'] = result['average_price'].round(2)

print(result)


   product_id  average_price
0           1           6.96
1           2          16.96


In [0]:
prices_df = spark.createDataFrame(prices)
prices_df.show(truncate=False)

+----------+-------------------+-------------------+-----+
|product_id|start_date         |end_date           |price|
+----------+-------------------+-------------------+-----+
|1         |2019-02-17 00:00:00|2019-02-28 00:00:00|5    |
|1         |2019-03-01 00:00:00|2019-03-22 00:00:00|20   |
|2         |2019-02-01 00:00:00|2019-02-20 00:00:00|15   |
|2         |2019-02-21 00:00:00|2019-03-31 00:00:00|30   |
+----------+-------------------+-------------------+-----+



In [0]:
units_sold_df = spark.createDataFrame(units_sold)
units_sold_df.show(truncate=False)

+----------+-------------------+-----+
|product_id|purchase_date      |units|
+----------+-------------------+-----+
|1         |2019-02-25 00:00:00|100  |
|1         |2019-03-01 00:00:00|15   |
|2         |2019-02-10 00:00:00|200  |
|2         |2019-03-22 00:00:00|30   |
+----------+-------------------+-----+



In [0]:
joined_df = prices_df.alias("p").join(
    units_sold_df.alias("u"),
    (F.col("p.product_id") == F.col("u.product_id")) &
    (F.col("u.purchase_date") >= F.col("p.start_date")) &
    (F.col("u.purchase_date") <= F.col("p.end_date")),
    how="inner"
)

# Calculate weighted average price
result_df = joined_df.groupBy("p.product_id") \
    .agg(F.round(F.sum(F.col("p.price") * F.col("u.units")) / F.sum("u.units"), 2).alias("average_price"))

result_df.show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+



In [0]:
prices_df.createOrReplaceTempView('prices')
units_sold_df.createOrReplaceTempView('unitsSold')

spark.sql('''
select prices.product_id, round(sum(price*units)/sum(units),2) as average_price 
from prices
inner join unitssold
on prices.product_id = unitssold.product_id
and (prices.start_date<=unitssold.purchase_date and prices.end_date>=unitssold.purchase_date)
group by prices.product_id;
''').show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+



[1075. Project Employees I](https://leetcode.com/problems/project-employees-i/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype(
    {'project_id': 'Int64', 'employee_id': 'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype(
    {'employee_id': 'Int64', 'name': 'object', 'experience_years': 'Int64'})

In [0]:
merged_df = pd.merge(project, employee, on='employee_id', how='inner')
result = merged_df.groupby('project_id')['experience_years'].mean().reset_index()
result.rename(columns={'experience_years':'average_years'}, inplace=True)
print(result)

   project_id  average_years
0           1            2.0
1           2            2.5


In [0]:
project_df = spark.createDataFrame(project)
project_df.show()

+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|         1|          1|
|         1|          2|
|         1|          3|
|         2|          1|
|         2|          4|
+----------+-----------+



In [0]:
employee_df = spark.createDataFrame(employee)
employee_df.show(truncate=False)

+-----------+------+----------------+
|employee_id|name  |experience_years|
+-----------+------+----------------+
|1          |Khaled|3               |
|2          |Ali   |2               |
|3          |John  |1               |
|4          |Doe   |2               |
+-----------+------+----------------+



In [0]:
project_df.join(employee_df, on=['employee_id'], how='inner').groupBy('project_id').agg(F.avg('experience_years').alias('average_years')).show()

+----------+-------------+
|project_id|average_years|
+----------+-------------+
|         1|          2.0|
|         2|          2.5|
+----------+-------------+



In [0]:
project_df.createOrReplaceTempView('project')
employee_df.createOrReplaceTempView('employee')

spark.sql('''
          SELECT project_id, round(avg(experience_years),2) AS average_years
          FROM project
          INNER JOIN employee on project.employee_id = employee.employee_id
          GROUP BY project.project_id;
          ''').show()

+----------+-------------+
|project_id|average_years|
+----------+-------------+
|         1|          2.0|
|         2|          2.5|
+----------+-------------+



[1633. Percentage of Users Attended a Contest](https://leetcode.com/problems/percentage-of-users-attended-a-contest/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id': 'Int64', 'user_name': 'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2],
        [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id': 'Int64', 'user_id': 'Int64'})

In [0]:
count_user = users['user_id'].nunique()

merged_df = register.merge(users,on='user_id',how='inner')

grouped = (
    merged_df.groupby('contest_id')['user_id'].count().reset_index(name='user_count')
)
grouped['percentage'] = round(100 * grouped['user_count'] / count_user ,2)
result = grouped[['contest_id','percentage']].sort_values(by=['percentage', 'contest_id'], ascending=[False, True])


In [0]:
result

,contest_id,percentage
1,208,100.00
2,209,100.00
3,210,100.00
4,215,66.67
0,207,33.33


In [0]:
users_df = spark.createDataFrame(users)
users_df.show(truncate=False)

+-------+---------+
|user_id|user_name|
+-------+---------+
|6      |Alice    |
|2      |Bob      |
|7      |Alex     |
+-------+---------+



In [0]:
register_df = spark.createDataFrame(register)
register_df.show(truncate=False)

+----------+-------+
|contest_id|user_id|
+----------+-------+
|215       |6      |
|209       |2      |
|208       |2      |
|210       |6      |
|208       |6      |
|209       |7      |
|209       |6      |
|215       |7      |
|208       |7      |
|210       |2      |
|207       |2      |
|210       |7      |
+----------+-------+



In [0]:
total_users = users_df.count()

result_df = (
    register_df.join(users_df, on=['user_id'], how='inner')
    .groupBy('contest_id')
    .agg(F.count('user_id').alias('user_count'))
    .withColumn('percentage', F.round(100 * F.col('user_count')/ total_users,2))
    .drop('user_count')
    .orderBy(F.col('percentage').desc(), 'contest_id')
).show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+



In [0]:
users_df.createOrReplaceTempView('users')
register_df.createOrReplaceTempView('register')

spark.sql(
        '''
        SELECT contest_id, 
                round(100 * count(register.user_id)/(select count(user_id) from users),2) as percentage
        FROM register
        INNER JOIN users
        on register.user_id = users.user_id
        GROUP BY contest_id
        ORDER BY percentage desc, contest_id asc;
        '''
).show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+



[1211. Queries Quality and Percentage](https://leetcode.com/problems/queries-quality-and-percentage/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [['Dog', 'Golden Retriever', 1, 5], ['Dog', 'German Shepherd', 2, 5], ['Dog', 'Mule', 200, 1],
        ['Cat', 'Shirazi', 5, 2], ['Cat', 'Siamese', 3, 3], ['Cat', 'Sphynx', 7, 4]]
queries = pd.DataFrame(data, columns=['query_name', 'result', 'position', 'rating']).astype(
    {'query_name': 'object', 'result': 'object', 'position': 'Int64', 'rating': 'Int64'})

In [0]:
queries['rating_by_position'] = queries['rating'] / queries['position']
quality_df = queries.groupby('query_name')['rating_by_position'].mean().round(2).reset_index()

# 2. Calculate poor_query_percentage: percentage of rating < 3 per query_name
queries['is_poor'] = queries['rating'] < 3
poor_df = queries.groupby('query_name')['is_poor'].mean().mul(100).round(2).reset_index()
poor_df.rename(columns={'is_poor': 'poor_query_percentage'}, inplace=True)

# 3. Merge the two results
result = pd.merge(quality_df, poor_df, on='query_name')

# Optional: Sort if needed
result = result.sort_values(by=['poor_query_percentage', 'query_name'], ascending=[False, True])

# Show final result
print(result)


  query_name  rating_by_position  poor_query_percentage
0        Cat                0.66                  33.33
1        Dog                 2.5                  33.33


In [0]:
queries_df = spark.createDataFrame(queries)
queries_df.show(truncate=False)

+----------+----------------+--------+------+------------------+-------+
|query_name|result          |position|rating|rating_by_position|is_poor|
+----------+----------------+--------+------+------------------+-------+
|Dog       |Golden Retriever|1       |5     |5.0               |false  |
|Dog       |German Shepherd |2       |5     |2.5               |false  |
|Dog       |Mule            |200     |1     |0.005             |true   |
|Cat       |Shirazi         |5       |2     |0.4               |true   |
|Cat       |Siamese         |3       |3     |1.0               |false  |
|Cat       |Sphynx          |7       |4     |0.5714285714285714|false  |
+----------+----------------+--------+------+------------------+-------+



In [0]:
queries_df \
    .groupBy('query_name') \
    .agg(
    F.round(F.avg(F.col('rating') / F.col('position')), 2).alias('quality'),
    F.round(F.avg(F.when(F.col('rating') < 3, 1).otherwise(0)) * 100, 2).alias('poor_query_percentage')
).show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                33.33|
|       Cat|   0.66|                33.33|
+----------+-------+---------------------+



In [0]:
queries_df.createOrReplaceTempView('queries')
spark.sql(
'''SELECT 
    q1.query_name, 
    ROUND(AVG(q1.rating / q1.position), 2) AS quality,
    ROUND(100.0 * SUM(CASE WHEN q2.rating < 3 THEN 1 ELSE 0 END) / COUNT(q2.rating), 2) AS poor_query_percentage
FROM queries q1
JOIN queries q2 ON q1.query_name = q2.query_name
GROUP BY q1.query_name;''').show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                33.33|
|       Cat|   0.66|                33.33|
+----------+-------+---------------------+



[1193. Monthly Transactions I](https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'],
        [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype(
    {'id': 'Int64', 'country': 'object', 'state': 'object', 'amount': 'Int64', 'trans_date': 'datetime64[ns]'})

In [0]:
# Step 1: Add month column
transactions['month'] = transactions['trans_date'].dt.to_period('M').astype(str)

# Step 2: Group by month and country to calculate all required metrics
summary_df = transactions.groupby(['month', 'country']).agg(
    trans_count=('id', 'count'),
    approved_count=('state', lambda x: (x == 'approved').sum()),
    trans_total_amount=('amount', 'sum'),
    approved_total_amount=('amount', lambda x: x[transactions.loc[x.index, 'state'] == 'approved'].sum())
).reset_index()

# View result
print(summary_df)


     month country  trans_count  approved_count  trans_total_amount  \
0  2018-12      US            2               1                3000   
1  2019-01      DE            1               1                2000   
2  2019-01      US            1               1                2000   

   approved_total_amount  
0                   1000  
1                   2000  
2                   2000  


In [0]:
transaction_df = spark.createDataFrame(transactions)
transaction_df.show()

+---+-------+--------+------+-------------------+
| id|country|   state|amount|         trans_date|
+---+-------+--------+------+-------------------+
|121|     US|approved|  1000|2018-12-18 00:00:00|
|122|     US|declined|  2000|2018-12-19 00:00:00|
|123|     US|approved|  2000|2019-01-01 00:00:00|
|124|     DE|approved|  2000|2019-01-07 00:00:00|
+---+-------+--------+------+-------------------+



In [0]:
transaction_df.withColumn('month',F.substring('trans_date',0,7))\
              .groupBy('month','country')\
              .agg(F.count('id').alias('trans_count')\
                   ,F.sum(F.when(F.col('state')=='approved',1).otherwise(0)).alias('approved_count')\
                   ,F.sum('amount').alias('trans_total_amount')\
                   ,F.sum(F.when(F.col('state') == 'approved', F.col('amount')).otherwise(0)).alias('approved_total_amount')).show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+



In [0]:
transaction_df.createTempView('transactions')

spark.sql('''
          select 
    left(trans_date,7) as month,
    country,
    count(id) as trans_count,
    sum(case when state='approved' then 1 else 0 end) as approved_count,
    sum(amount) as trans_total_amount,
    sum(case when state='approved' then amount else 0 end) as approved_total_amount
from transactions
group by month, country;
          ''').show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+



[1174. Immediate Food Delivery II](https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'],
        [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'],
        [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data,
                        columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype(
    {'delivery_id': 'Int64', 'customer_id': 'Int64', 'order_date': 'datetime64[ns]',
     'customer_pref_delivery_date': 'datetime64[ns]'})

In [0]:
first_orders = delivery.sort_values(by=['customer_id','order_date']).drop_duplicates(subset='customer_id',keep='first')
immediate_matches = (first_orders['order_date'] == first_orders['customer_pref_delivery_date']).sum()
total_first_orders = len(first_orders)
immediate_percentage = round((immediate_matches / total_first_orders) * 100,2)

In [0]:
immediate_percentage

Out[67]: 50.0

In [0]:
delivery_df = spark.createDataFrame(delivery)
delivery_df.show(truncate=False)

+-----------+-----------+-------------------+---------------------------+
|delivery_id|customer_id|order_date         |customer_pref_delivery_date|
+-----------+-----------+-------------------+---------------------------+
|1          |1          |2019-08-01 00:00:00|2019-08-02 00:00:00        |
|2          |2          |2019-08-02 00:00:00|2019-08-02 00:00:00        |
|3          |1          |2019-08-11 00:00:00|2019-08-12 00:00:00        |
|4          |3          |2019-08-24 00:00:00|2019-08-24 00:00:00        |
|5          |3          |2019-08-21 00:00:00|2019-08-22 00:00:00        |
|6          |2          |2019-08-11 00:00:00|2019-08-13 00:00:00        |
|7          |4          |2019-08-09 00:00:00|2019-08-09 00:00:00        |
+-----------+-----------+-------------------+---------------------------+



In [0]:
from pyspark.sql import Window
delivery_df \
    .withColumn('rn', F.row_number().over(Window.partitionBy('customer_id').orderBy('order_date'))) \
    .filter(F.col('rn') == 1) \
    .agg(
    F.round(100*
        F.sum(F.when(F.col('order_date') == F.col('customer_pref_delivery_date'), 1).otherwise(0)) / F.count('*'),
        2) \
        .alias('immediate_percentage')
) \
    .show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+



In [0]:
delivery_df.createOrReplaceTempView('delivery')

spark.sql('''
with tbl as (
select *, row_number() over (partition by customer_id order by order_date) as rn
from delivery
)
select 
    round(
    100*sum(case when order_date=customer_pref_delivery_date then 1 else 0 end)/count(*),
    2) as immediate_percentage
from tbl
where rn = 1;
''').show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+



[550. Game Play Analysis IV](https://leetcode.com/problems/game-play-analysis-iv/description/?envType=study-plan-v2&envId=top-sql-50)

In [0]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0],
        [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype(
    {'player_id': 'Int64', 'device_id': 'Int64', 'event_date': 'datetime64[ns]', 'games_played': 'Int64'})

In [0]:
first_login_df = activity.sort_values(by=['player_id', 'event_date']).drop_duplicates(subset='player_id', keep='first')

# Step 2: Merge first login back with full data to get next day's record
merged = pd.merge(first_login_df[['player_id', 'event_date']], activity, on='player_id')

# Step 3: Check if any event_date in full data is exactly one day after first login
merged['is_next_day'] = merged['event_date_y'] == merged['event_date_x'] + pd.Timedelta(days=1)

# Step 4: For each player, check if at least one next-day login exists
next_day_login = merged.groupby('player_id')['is_next_day'].any().reset_index()

# Step 5: Compute the fraction
fraction = round(next_day_login['is_next_day'].sum() / len(next_day_login), 2)

# Step 6: Display as DataFrame
result = pd.DataFrame({'fraction': [fraction]})
print(result)

   fraction
0      0.33


In [0]:
activity_df = spark.createDataFrame(activity)
activity_df.show(truncate=False)

+---------+---------+-------------------+------------+
|player_id|device_id|event_date         |games_played|
+---------+---------+-------------------+------------+
|1        |2        |2016-03-01 00:00:00|5           |
|1        |2        |2016-03-02 00:00:00|6           |
|2        |3        |2017-06-25 00:00:00|1           |
|3        |1        |2016-03-02 00:00:00|0           |
|3        |4        |2018-07-03 00:00:00|5           |
+---------+---------+-------------------+------------+



In [0]:
activity_df.withColumn('next_date',
                       F.lead('event_date').over(Window.partitionBy('player_id').orderBy('event_date'))
                       ) \
    .withColumn('rn', F.row_number().over(Window.partitionBy('player_id').orderBy('event_date'))) \
    .filter(F.col('rn') == 1) \
    .agg(
    F.round(
        F.sum(F.when(F.datediff(F.col('next_date'), F.col('event_date')) == 1, 1).otherwise(0)) / F.count('player_id')
        , 2).alias('fraction')
) \
    .show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [0]:
activity_df.createOrReplaceTempView('activity')

spark.sql('''
with tbl as (
    select 
        *,
        lead(event_date) over (partition by player_id order by event_date) as next_date,
        row_number() over (partition by player_id order by event_date) as rn
    from activity
)
select 
    round(
    sum(case when datediff(next_date,event_date)=1 then 1 else 0 end)/count(*)
    ,2) as fraction 
from tbl
where rn=1;
''').show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+



In [0]:
spark.stop()